In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-96gvyawf
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-96gvyawf
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=9f584124507368219c6dfa07f7ac02920b38ffa90d7f40004e881295f48f0415
  Stored in directory: /tmp/pip-ephem-wheel-cache-ji6yweav/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


**Load the Dataset and unzip it**

In [ ]:
!rm -r *
!wget -q https://www.dropbox.com/s/tfldi58ch27lzse/Lab6.zip
!unzip Lab6.zip
!rm Lab6.zip
!ls

Archive:  Lab6.zip
   creating: Dataset/Test/
   creating: Dataset/Test/0/
 extracting: Dataset/Test/0/input0.ppm  
  inflating: Dataset/Test/0/input1.raw  
  inflating: Dataset/Test/0/Myoutput.ppm  
  inflating: Dataset/Test/0/output.ppm  
   creating: Dataset/Test/1/
  inflating: Dataset/Test/1/input0.ppm  
  inflating: Dataset/Test/1/input1.raw  
  inflating: Dataset/Test/1/Myoutput.ppm  
  inflating: Dataset/Test/1/output.ppm  
   creating: Dataset/Test/2/
  inflating: Dataset/Test/2/input0.ppm  
  inflating: Dataset/Test/2/input1.raw  
  inflating: Dataset/Test/2/Myoutput.ppm  
  inflating: Dataset/Test/2/output.ppm  
   creating: Dataset/Test/3/
 extracting: Dataset/Test/3/input0.ppm  
  inflating: Dataset/Test/3/input1.raw  
  inflating: Dataset/Test/3/Myoutput.ppm  
  inflating: Dataset/Test/3/output.ppm  
   creating: Dataset/Test/4/
 extracting: Dataset/Test/4/input0.ppm  
  inflating: Dataset/Test/4/input1.raw  
  inflating: Dataset/Test/4/Myoutput.ppm  
  inflating: Dataset

# **CUDA Convolution Code Using Global Memory**

Code to be modified where there are TODO indications

In [ ]:
%%writefile lab6-GLOBALM.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

#define Mask_width 5
#define Mask_radius Mask_width / 2
#define clamp(x) (min(max((x), 0.0), 1.0))
typedef struct {
    unsigned char red, green, blue;
} PPMPixel;

#define RGB_COMPONENT_COLOR 255
#define CREATOR "AMMI Lab"
PPMPixel* readPPM(const char* filename, int *width, int *height,int *channel)
{
    char buff[16];
   PPMPixel* img;
    FILE* fp;
    int c;
	printf("Reading %s\n",filename);
    //open PPM file for reading
    fp = fopen(filename, "rb");
    if (!fp) {
        printf("Unable to open file '%s'\n", filename);
        exit(1);
    }

    //read image format
    if (!fgets(buff, sizeof(buff), fp)) {
		printf("wrong format\n");
        perror(filename);
        exit(1);
    }

    //check the image format
    if (buff[0] != 'P' || buff[1] != '6') {
        printf("Invalid image format (must be 'P6')\n");
        exit(1);
    }

    //check for comments
    c = getc(fp);
    while (c == '#') {
        while (getc(fp) != '\n');
        c = getc(fp);
    }

    ungetc(c, fp);
    //read image size information
    if (fscanf(fp, "%d %d", width, height) != 2) {
        printf("Invalid image size (error loading '%s')\n", filename);
        exit(1);
    }
    //read rgb component
    if (fscanf(fp, "%d", channel) != 1) {
        printf("Invalid rgb component (error loading '%s')\n", filename);
        exit(1);
    }

    //check rgb component depth
    if (*channel != RGB_COMPONENT_COLOR) {
        printf("'%s' does not have 8-bits components\n", filename);
        exit(1);
    }
	*channel=3;
    while (fgetc(fp) != '\n');
    //memory allocation for pixel data
    img = (PPMPixel*)malloc(*width * *height * sizeof(PPMPixel));

    if (!img) {
        printf("Unable to allocate memory\n");
        exit(1);
    }

    //read pixel data from file
    if (fread(img, 3*(*width)*sizeof(unsigned char), (*height), fp) != (*height)) {
        printf("Error loading image '%s'\n", filename);
        exit(1);
    }

    fclose(fp);
    return img;
}
void writePPM(const char* filename, PPMPixel *img,int width,int height)
{
    FILE* fp;
    //open file for output
    fp = fopen(filename, "wb");
    if (!fp) {
        printf("Unable to open file '%s'\n", filename);
        exit(1);
    }

    //write the header file
    //image format
    fprintf(fp, "P6\n");

    //comments
    fprintf(fp, "# Created by %s\n", CREATOR);

    //image size
    fprintf(fp, "%d %d\n", width, height);

    // rgb component depth
    fprintf(fp, "%d\n", RGB_COMPONENT_COLOR);

    // pixel data
    fwrite(img, 3*width, height, fp);
    fclose(fp);
}
float* ImportMatrix(char* filename, int* N, int* M)
{
    FILE* handle;
    float* val;
    int i, j;
    printf("Reading File %s\n", filename);
    if (filename == NULL) {
        return NULL;
    }

    handle = fopen(filename, "r");
    if (handle == NULL) {
        printf("Error opening file: %s\n", filename);
        return NULL;
    }
    fscanf(handle, "%d %d", N, M);
    val = (float*)malloc(*N * *M * sizeof(float));
    for (i = 0; i < *N; i++) {
        for (j = 0; j < *M; j++) {
            fscanf(handle, "%f ", val + (i * *M) + j);
        }
    }
    fclose(handle);
    return val;
}
float* Convert_PPM_to_float(PPMPixel *ImageData, int Width, int Height) {
    float* Fimage;
    int i,j=0;
    int numPixels;
    numPixels = Width * Height;
    Fimage = (float*)malloc(numPixels * sizeof(float)*3);
        for (i = 0; i < numPixels; i++) {
                Fimage[j] = ((float)ImageData[i].red)/255.;
                Fimage[j +1] = ((float)ImageData[i].green)/255.;
                Fimage[j +2] = ((float)ImageData[i].blue)/255.;
                j = j + 3;
        }
        return Fimage;
}
PPMPixel* Convert_Float_to_PPM(float* ImageData, int Width, int Height) {
    PPMPixel *PPMimage;
    int i,j=0,numPixels;
    numPixels = Width * Height;
    PPMimage = (PPMPixel*)malloc(numPixels * sizeof(PPMPixel));
    if (PPMimage) {
        for (i = 0; i < numPixels; i++) {
            PPMimage[i].red = (unsigned char)(ImageData[j]*255);
            PPMimage[i].green = (unsigned char)(ImageData[j + 1]*255);
            PPMimage[i].blue = (unsigned char)(ImageData[j + 2]*255);
            j = j + 3;

        }
    }
    return PPMimage;
}

__global__ void convolution(float *I, const float *M, float *P, int channels, int width, int height) {


	int Col = blockIdx.x * blockDim.x + threadIdx.x;
	int Row = blockIdx.y * blockDim.y + threadIdx.y;
	// The code below is adapted from the basic 2D convolution kernel provided in the course notes.
	// The adaptation required is to handle more than a single channel (in this case, up to k channels).
	// TODO: Convolution using global memory

  if (Col < width && Row < height) {

    for (int c = 0; c < channels; c++) {
      float pixVal = 0;

      int N_start_col = Col - Mask_radius;
      int N_start_row = Row - Mask_radius;

      for (int j = 0; j < Mask_width; ++j) {
        for (int k = 0; k < Mask_width; ++k) {
          int curRow = N_start_row + j;
          int curCol = N_start_col + k;

          if (curRow > -1 && curRow < height && curCol > -1 && curCol < width) {
            pixVal += I[ (curRow * width + curCol) * channels + c ] * M[ j * Mask_width + k ];
          }

        }
      }
      P[ (Row * width + Col) * channels + c ] = clamp(pixVal);
    }

  }

}




clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000.0); }
	return 0;
}

int main(int argc, char **argv) {
	int maskRows;
	int maskColumns;
	int imageChannels;
	int imageWidth;
	int imageHeight;
  int RimageChannels;
  int RimageWidth;
  int RimageHeight;
  PPMPixel *inputImage;
  PPMPixel *RImage;
  PPMPixel *outputImage;
	float *hostInputImageData;
  float* hostRImageData;
	float *hostOutputImageData;
	float *hostMaskData;
	float *deviceInputImageData;
	float *deviceOutputImageData;
	float *deviceMaskData;
  int i,prt = 0;
	cudaEvent_t start, stop;
	float gpu_time = 0.0f;
  float meanDiff = 0;
	int ti;
	clock_t StartingTime;
  char argv1[50],argv2[50],argv3[50],argv4[50],argv5[3];

for (ti=0;ti<9;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/Test/%i/output.ppm",ti);
  sprintf(argv2,"Dataset/Test/%i/input0.ppm",ti);
	sprintf(argv3,"Dataset/Test/%i/input1.raw",ti);
	sprintf(argv4,"Dataset/Test/%i/Myoutput.ppm",ti);
	sprintf(argv5,"ON");

	printf("Running GPU Convolution Global Memory Version...\n");
	if (strcmp(argv5, "ON") == 0)prt = 1;

  StartingTime = Time_start();
	printf("Reading image data \n");
	RImage = (PPMPixel *)readPPM(argv1, &RimageWidth, &RimageHeight,&RimageChannels);
  inputImage = (PPMPixel *)readPPM(argv2, &imageWidth, &imageHeight,&imageChannels);
  printf("Input Image Width %d Height %d Channels %d\n", imageWidth, imageHeight, imageChannels);
	printf("RInput Image Width %d Height %d Channels %d\n", RimageWidth, RimageHeight, RimageChannels);
  hostMaskData = (float *) ImportMatrix(argv3,&maskRows,&maskColumns);
  printf("Convolution Operator Size %d %d\n", maskRows, maskColumns);
  Elapsed_time(StartingTime, "Reading Data.", prt);

  StartingTime = Time_start();

  hostInputImageData = (float*)Convert_PPM_to_float(inputImage, imageWidth, imageHeight);
  hostRImageData= (float*)Convert_PPM_to_float(RImage, RimageWidth, RimageHeight);
	hostOutputImageData = (float*)Convert_PPM_to_float(RImage, RimageWidth, RimageHeight);

	Elapsed_time(StartingTime, "Convert pixel data to float", prt);

	/*TODO: Allocate GPU Memory*/

	StartingTime=Time_start();

  cudaMalloc(&deviceInputImageData, imageWidth * imageHeight * imageChannels * sizeof(float));
  cudaMalloc(&deviceOutputImageData, imageWidth * imageHeight * imageChannels * sizeof(float));
  cudaMalloc(&deviceMaskData, maskColumns * maskRows * sizeof(float));

	Elapsed_time(StartingTime, "Allocate GPU Memory", prt);

	StartingTime=Time_start();

	// TODO: Copy memory to the GPU here
  cudaMemcpy(deviceInputImageData, hostInputImageData, imageWidth * imageHeight * imageChannels * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(deviceMaskData, hostMaskData, maskColumns * maskRows * sizeof(float), cudaMemcpyHostToDevice);

	Elapsed_time(StartingTime, "Copying input memory to the GPU.", prt);

	// Launch kernel
	// ----------------------------------------------------------
	printf( "Launching kernel\n");

	// TODO: Perform kernel computation here

	dim3 block(16, 16, 1);
	dim3 grid(ceil((float)imageWidth / 16), ceil((float)imageHeight / 16), 1);
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
	convolution << <grid, block >> >(deviceInputImageData, deviceMaskData, deviceOutputImageData, imageChannels, imageWidth, imageHeight);
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	printf("-------->Convolution on Global Memory Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);

	StartingTime = Time_start();

	// TODO: Copy the GPU memory back to the CPU here
  cudaMemcpy(hostOutputImageData, deviceOutputImageData, imageWidth * imageHeight * imageChannels * sizeof(float), cudaMemcpyDeviceToHost);

	Elapsed_time(StartingTime, "Copying output memory to the CPU.", prt);

	StartingTime = Time_start();
	// TODO: Free the GPU memory here
  cudaFree(deviceOutputImageData);
  cudaFree(deviceInputImageData);
  cudaFree(deviceMaskData);

	Elapsed_time(StartingTime, "Freeing GPU Memory.", prt);

	// Verify correctness
	// -----------------------------------------------------
	outputImage =Convert_Float_to_PPM(hostOutputImageData, imageWidth, imageHeight);
  // -----------------------------------------------------
  for (i = 0; i < imageWidth*imageHeight*imageChannels; i++) {
      meanDiff = meanDiff + fabs(hostRImageData[i] - hostOutputImageData[i]);
  }
  meanDiff = meanDiff / ((float)(imageWidth * imageHeight * imageChannels));
  if (meanDiff > 0.1) {
      printf("%f Failed\n", meanDiff);

  }
  else {
      printf("%f Passed\n",meanDiff);
  }

    // Save Convoluted Image
    printf("Saving Convoluted image %s %d %d\n", argv4, imageWidth, imageHeight);
    writePPM(argv4,outputImage, imageWidth, imageHeight);

	// Free host memory
	free(hostMaskData);
	free(outputImage);
	free(inputImage);
  free(RImage);
  free(hostInputImageData);
  free(hostRImageData);
  free(hostOutputImageData);

}

	return 1;
}


Writing lab6-GLOBALM.cu


**Compile GPU Code Using Global Memory**

In [ ]:
!nvcc  lab6-GLOBALM.cu -o lab6-GLOBALM.exe
!ls

Dataset  lab6-GLOBALM.cu  lab6-GLOBALM.exe


**Execute GPU Code Using Global Memory**

In [ ]:
!./lab6-GLOBALM.exe



 +++++++++++++Test 0
Running GPU Convolution Global Memory Version...
Reading image data 
Reading Dataset/Test/0/output.ppm
Reading Dataset/Test/0/input0.ppm
Input Image Width 64 Height 64 Channels 3
RInput Image Width 64 Height 64 Channels 3
Reading File Dataset/Test/0/input1.raw
Convolution Operator Size 5 5
Reading Data. Elapsed Time 0.151000 in mili-seconds
Convert pixel data to float Elapsed Time 0.251000 in mili-seconds
Allocate GPU Memory Elapsed Time 296.710000 in mili-seconds
Copying input memory to the GPU. Elapsed Time 0.064000 in mili-seconds
Launching kernel
-------->Convolution on Global Memory Kernel Elapsed Time*********** 0.037056 in mili-seconds
Copying output memory to the CPU. Elapsed Time 0.046000 in mili-seconds
Freeing GPU Memory. Elapsed Time 0.139000 in mili-seconds
0.001961 Passed
Saving Convoluted image Dataset/Test/0/Myoutput.ppm 64 64


 +++++++++++++Test 1
Running GPU Convolution Global Memory Version...
Reading image data 
Reading Dataset/Test/1/output.

Save trace into Lab6GlobalM.txt

# **GPU Convolution Code Using Shared Memory**

In [ ]:
%%writefile lab6-TILED.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

#define Mask_width 5
#define Mask_radius Mask_width / 2
#define TILE_WIDTH 16
#define w (TILE_WIDTH + Mask_width - 1)

#define clamp(x) (min(max((x), 0.0), 1.0))
typedef struct {
    unsigned char red, green, blue;
} PPMPixel;

#define RGB_COMPONENT_COLOR 255
#define CREATOR "AMMI Lab"
PPMPixel* readPPM(const char* filename, int *width, int *height,int *channel)
{
    char buff[16];
   PPMPixel* img;
    FILE* fp;
    int c;
	printf("Reading %s\n",filename);
    //open PPM file for reading
    fp = fopen(filename, "rb");
    if (!fp) {
        printf("Unable to open file '%s'\n", filename);
        exit(1);
    }

    //read image format
    if (!fgets(buff, sizeof(buff), fp)) {
		printf("wrong format\n");
        perror(filename);
        exit(1);
    }

    //check the image format
    if (buff[0] != 'P' || buff[1] != '6') {
        printf("Invalid image format (must be 'P6')\n");
        exit(1);
    }

    //check for comments
    c = getc(fp);
    while (c == '#') {
        while (getc(fp) != '\n');
        c = getc(fp);
    }

    ungetc(c, fp);
    //read image size information
    if (fscanf(fp, "%d %d", width, height) != 2) {
        printf("Invalid image size (error loading '%s')\n", filename);
        exit(1);
    }
    //read rgb component
    if (fscanf(fp, "%d", channel) != 1) {
        printf("Invalid rgb component (error loading '%s')\n", filename);
        exit(1);
    }

    //check rgb component depth
    if (*channel != RGB_COMPONENT_COLOR) {
        printf("'%s' does not have 8-bits components\n", filename);
        exit(1);
    }
	*channel=3;
    while (fgetc(fp) != '\n');
    //memory allocation for pixel data
    img = (PPMPixel*)malloc(*width * *height * sizeof(PPMPixel));

    if (!img) {
        printf("Unable to allocate memory\n");
        exit(1);
    }

    //read pixel data from file
    if (fread(img, 3*(*width)*sizeof(unsigned char), (*height), fp) != (*height)) {
        printf("Error loading image '%s'\n", filename);
        exit(1);
    }

    fclose(fp);
    return img;
}
void writePPM(const char* filename, PPMPixel *img,int width,int height)
{
    FILE* fp;
    //open file for output
    fp = fopen(filename, "wb");
    if (!fp) {
        printf("Unable to open file '%s'\n", filename);
        exit(1);
    }

    //write the header file
    //image format
    fprintf(fp, "P6\n");

    //comments
    fprintf(fp, "# Created by %s\n", CREATOR);

    //image size
    fprintf(fp, "%d %d\n", width, height);

    // rgb component depth
    fprintf(fp, "%d\n", RGB_COMPONENT_COLOR);

    // pixel data
    fwrite(img, 3*width, height, fp);
    fclose(fp);
}
float* ImportMatrix(char* filename, int* N, int* M)
{
    FILE* handle;
    float* val;
    int i, j;
    printf("Reading File %s\n", filename);
    if (filename == NULL) {
        return NULL;
    }

    handle = fopen(filename, "r");
    if (handle == NULL) {
        printf("Error opening file: %s\n", filename);
        return NULL;
    }
    fscanf(handle, "%d %d", N, M);
    val = (float*)malloc(*N * *M * sizeof(float));
    for (i = 0; i < *N; i++) {
        for (j = 0; j < *M; j++) {
            fscanf(handle, "%f ", val + (i * *M) + j);
        }
    }
    fclose(handle);
    return val;
}
float* Convert_PPM_to_float(PPMPixel *ImageData, int Width, int Height) {
    float* Fimage;
    int i,j=0;
    int numPixels;
    numPixels = Width * Height;
    Fimage = (float*)malloc(numPixels * sizeof(float)*3);
        for (i = 0; i < numPixels; i++) {
                Fimage[j] = ((float)ImageData[i].red)/255.;
                Fimage[j +1] = ((float)ImageData[i].green)/255.;
                Fimage[j +2] = ((float)ImageData[i].blue)/255.;
                j = j + 3;
        }
        return Fimage;
}
PPMPixel* Convert_Float_to_PPM(float* ImageData, int Width, int Height) {
    PPMPixel *PPMimage;
    int i,j=0,numPixels;
    numPixels = Width * Height;
    PPMimage = (PPMPixel*)malloc(numPixels * sizeof(PPMPixel));
    if (PPMimage) {
        for (i = 0; i < numPixels; i++) {
            PPMimage[i].red = (unsigned char)(ImageData[j]*255);
            PPMimage[i].green = (unsigned char)(ImageData[j + 1]*255);
            PPMimage[i].blue = (unsigned char)(ImageData[j + 2]*255);
            j = j + 3;

        }
    }
    return PPMimage;
}

__global__ void convolution_tiled(float *I, const float *M, float *P, int channels, int width, int height) {
  // Reference: Textbook, Chapter 7 on eclass.
	__shared__ float N_ds[w][w];
	int i, j, k;

  int tx = threadIdx.x;
  int ty = threadIdx.y;
  int row_o = blockIdx.y * TILE_WIDTH + ty;
  int col_o = blockIdx.x * TILE_WIDTH + tx;
  int row_i = row_o - Mask_radius;
  int col_i = col_o - Mask_radius;

	for (k = 0; k < channels; k++) {
		// TODO: First batch loading
    if ((row_i >= 0) && (row_i < height) && (col_i >= 0) && (col_i < width)) {
      N_ds[ty][tx] = I[ (row_i * width + col_i) * channels + k ];
    } else{
      N_ds[ty][tx] = 0.0f;
    }

		// TODO: Second batch loading
    /*
    // Second batch loading code is commented for better performance;
    // Uncomment if needed:
    if (ty < w) {
      if ((row_i >= 0) && (row_i < height) && (col_i >= 0) && (col_i < width)) {
        N_ds[ty][tx] = I[ (row_i * width + col_i) * channels + k ];
      } else{
        N_ds[ty][tx] = 0.0f;
      }
    }
    */
		__syncthreads();

    //TODO: Compute the convolution
    float pixVal = 0.0f;
    if (ty < TILE_WIDTH && tx < TILE_WIDTH) {
      for (i = 0; i < Mask_width; i++) {
        for (j = 0; j < Mask_width; j++) {
          pixVal += M[i * Mask_width + j] * N_ds[i + ty][j + tx];
        }
      }
      if (row_o < height && col_o < width) {
        P[ (row_o * width + col_o) * channels + k ] = clamp(pixVal);
      }
		}
		__syncthreads();
	}
}


clock_t Time_start() {
	clock_t StartingTime;
	StartingTime=clock();
	return StartingTime;
}
int Elapsed_time(clock_t start, const char* message, int prt) {
clock_t end;
double cpu_time_used;
end = clock();
cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
	if (prt == 1) { printf("%s Elapsed Time %f in mili-seconds\n", message, cpu_time_used*1000.0); }
	return 0;
}

int main(int argc, char **argv) {
	int maskRows;
	int maskColumns;
	int imageChannels;
	int imageWidth;
	int imageHeight;
  int RimageChannels;
  int RimageWidth;
  int RimageHeight;
  PPMPixel *inputImage;
  PPMPixel *RImage;
  PPMPixel *outputImage;
	float *hostInputImageData;
  float* hostRImageData;
	float *hostOutputImageData;
	float *hostMaskData;
	float *deviceInputImageData;
	float *deviceOutputImageData;
	float *deviceMaskData;
  int i,prt = 0;
	cudaEvent_t start, stop;
	float gpu_time = 0.0f;
  float meanDiff = 0;
	int ti;
	clock_t StartingTime;
  char argv1[50],argv2[50],argv3[50],argv4[50],argv5[3];

for (ti=0;ti<9;ti++){
	printf("\n\n +++++++++++++Test %i\n",ti);
	sprintf(argv1,"Dataset/Test/%i/output.ppm",ti);
  sprintf(argv2,"Dataset/Test/%i/input0.ppm",ti);
	sprintf(argv3,"Dataset/Test/%i/input1.raw",ti);
	sprintf(argv4,"Dataset/Test/%i/Myoutput.ppm",ti);
	sprintf(argv5,"ON");

	printf("Running GPU Convolution Tiled Version...\n");
	if (strcmp(argv5, "ON") == 0)prt = 1;

  StartingTime = Time_start();
	printf("Reading image data \n");
	RImage = (PPMPixel *)readPPM(argv1, &RimageWidth, &RimageHeight,&RimageChannels);
  inputImage = (PPMPixel *)readPPM(argv2, &imageWidth, &imageHeight,&imageChannels);
  printf("Input Image Width %d Height %d Channels %d\n", imageWidth, imageHeight, imageChannels);
	printf("RInput Image Width %d Height %d Channels %d\n", RimageWidth, RimageHeight, RimageChannels);
  hostMaskData = (float *) ImportMatrix(argv3,&maskRows,&maskColumns);
  printf("Convolution Operator Size %d %d\n", maskRows, maskColumns);
  Elapsed_time(StartingTime, "Reading Data.", prt);

  // TODO: Allocate the hostC matrix

  StartingTime = Time_start();

  cudaMalloc(&deviceInputImageData, imageWidth * imageHeight * imageChannels * sizeof(float));
  cudaMalloc(&deviceOutputImageData, imageWidth * imageHeight * imageChannels * sizeof(float));
  cudaMalloc(&deviceMaskData, maskColumns * maskRows * sizeof(float));

  hostInputImageData = (float*)Convert_PPM_to_float(inputImage, imageWidth, imageHeight);
  hostRImageData= (float*)Convert_PPM_to_float(RImage, RimageWidth, RimageHeight);
	hostOutputImageData = (float*)Convert_PPM_to_float(RImage, RimageWidth, RimageHeight);

	Elapsed_time(StartingTime, "Convert pixel data to float", prt);

	/*TODO: Allocate GPU Memory*/

	StartingTime=Time_start();




	Elapsed_time(StartingTime, "Allocate GPU Memory", prt);

	StartingTime=Time_start();

	// TODO: Copy memory to the GPU here
  cudaMemcpy(deviceInputImageData, hostInputImageData, imageWidth * imageHeight * imageChannels * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(deviceMaskData, hostMaskData, maskColumns * maskRows * sizeof(float), cudaMemcpyHostToDevice);

	Elapsed_time(StartingTime, "Copying input memory to the GPU.", prt);

	// Launch kernel
	// ----------------------------------------------------------
	printf( "Launching kernel\n");

	// TODO: Perform kernel computation here

	dim3 block(16, 16, 1);
	dim3 grid(ceil((float)imageWidth / 16), ceil((float)imageHeight / 16), 1);
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
	convolution_tiled << <grid, block >> >(deviceInputImageData, deviceMaskData, deviceOutputImageData, imageChannels, imageWidth, imageHeight);
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	printf("-------->Convolution on Global Memory Kernel Elapsed Time*********** %f in mili-seconds\n", gpu_time);

	StartingTime = Time_start();

	// TODO: Copy the GPU memory back to the CPU here
  cudaMemcpy(hostOutputImageData, deviceOutputImageData, imageWidth * imageHeight * imageChannels * sizeof(float), cudaMemcpyDeviceToHost);


	Elapsed_time(StartingTime, "Copying output memory to the CPU.", prt);

	StartingTime = Time_start();
	// TODO: Free the GPU memory here
  cudaFree(deviceOutputImageData);
  cudaFree(deviceInputImageData);
  cudaFree(deviceMaskData);

	Elapsed_time(StartingTime, "Freeing GPU Memory.", prt);

	// Verify correctness
	// -----------------------------------------------------
	outputImage =Convert_Float_to_PPM(hostOutputImageData, imageWidth, imageHeight);
  // -----------------------------------------------------
  for (i = 0; i < imageWidth*imageHeight*imageChannels; i++) {
      meanDiff = meanDiff + fabs(hostRImageData[i] - hostOutputImageData[i]);
  }
  meanDiff = meanDiff / ((float)(imageWidth * imageHeight * imageChannels));
  if (meanDiff > 0.1) {
      printf("%f Failed\n", meanDiff);

  }
  else {
      printf("%f Passed\n",meanDiff);
  }

    // Save Convoluted Image
    printf("Saving Convoluted image %s %d %d\n", argv4, imageWidth, imageHeight);
    writePPM(argv4,outputImage, imageWidth, imageHeight);

	// Free host memory
	free(hostMaskData);
	free(outputImage);
	free(inputImage);
  free(RImage);
  free(hostInputImageData);
  free(hostRImageData);
  free(hostOutputImageData);

}

	return 1;
}


Writing lab6-TILED.cu


**Compile CUDA Convolution code using shared memory**

In [ ]:
!nvcc  lab6-TILED.cu -o lab6-TILED.exe
!ls

Dataset  lab6-GLOBALM.cu  lab6-GLOBALM.exe  lab6-TILED.cu  lab6-TILED.exe


**Execute CUDA Convolution code using shared memory**

In [ ]:
!./lab6-TILED.exe



 +++++++++++++Test 0
Running GPU Convolution Tiled Version...
Reading image data 
Reading Dataset/Test/0/output.ppm
Reading Dataset/Test/0/input0.ppm
Input Image Width 64 Height 64 Channels 3
RInput Image Width 64 Height 64 Channels 3
Reading File Dataset/Test/0/input1.raw
Convolution Operator Size 5 5
Reading Data. Elapsed Time 0.111000 in mili-seconds
Convert pixel data to float Elapsed Time 282.973000 in mili-seconds
Allocate GPU Memory Elapsed Time 0.001000 in mili-seconds
Copying input memory to the GPU. Elapsed Time 0.053000 in mili-seconds
Launching kernel
-------->Convolution on Global Memory Kernel Elapsed Time*********** 0.029312 in mili-seconds
Copying output memory to the CPU. Elapsed Time 0.033000 in mili-seconds
Freeing GPU Memory. Elapsed Time 0.115000 in mili-seconds
0.064783 Passed
Saving Convoluted image Dataset/Test/0/Myoutput.ppm 64 64


 +++++++++++++Test 1
Running GPU Convolution Tiled Version...
Reading image data 
Reading Dataset/Test/1/output.ppm
Reading Data

Save trace into Lab6Tiled.txt